In [1]:
!pip install --upgrade python-terrier

!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     |████████████████████████████████| 97 kB 3.9 MB/s 
     |████████████████████████████████| 1.1 MB 60.0 MB/s 
     |████████████████████████████████| 69 kB 9.4 MB/s 
     |████████████████████████████████| 255 kB 69.0 MB/s 
     |████████████████████████████████| 41 kB 394 kB/s 
     |████████████████████████████████| 6.4 MB 59.0 MB/s 
     |████████████████████████████████| 126 kB 73.4 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
     |████████████████████████████████| 294 kB 69.0 MB/s 
     |████████████████████████████████| 1.8 MB 53.4 MB/s 
     |████████████████████████████████| 72 kB 1.9 MB/s 
     |████████████████████████████████| 291 kB 79.5 MB/s 
     |████████████████████████████████| 59 kB 9.0 MB/s 
     |████████████████████████████████| 45 kB 4.0 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=d2a21ea4a10c32e2f957a217457a7f0ccdcb8cd680a232a4c3af62a54cfe4d2d
  Stored in directory: /root/.cac

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
#libraries

import pyterrier as pt
import string 
from nltk.stem.porter import *
from nltk.tokenize import WordPunctTokenizer
import pandas as pd

In [4]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation+'‐'+'′'+'–'+'‘'+'’'+'“'+'”'])
    return punctuationfree

import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def tokenization(text):
  tokenized_text = WordPunctTokenizer().tokenize(text)
  return tokenized_text

def remove_stopwords(text):
  tokenized_text = WordPunctTokenizer().tokenize(text)
  tokenized_text_without_stopwords = []
  for token in tokenized_text:
    if token.lower() not in stop_words:
      tokenized_text_without_stopwords.append(token)
  return " ".join(tokenized_text_without_stopwords)

def porter_stemmer(text):
  stemmer = PorterStemmer()
  return_text = stemmer.stem(text)

  return return_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#load datasets and init pyterrier

if not pt.started():
  pt.init()

root_dir = '/content/gdrive/MyDrive'
base_dir = root_dir + '/Progetto IR/data'
#base_dir = root_dir + '/IR_Dataset'

collection = pd.read_pickle(base_dir + "/collection.pkl")
qrels = pd.read_pickle(base_dir + "/qrels.pkl")
queries = pd.read_pickle(base_dir + "/queries.pkl")

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [6]:
pt.set_property("termpipelines", "")
pt.set_property("termpipelines", "Stopwords")
#pt.set_property("termpipelines", "PorterStemmer")

pd_indexer = pt.DFIndexer("/pd_index", overwrite = True)
indexref = pd_indexer.index(collection["text"], collection["docno"], collection["date"], collection["title"])

17:49:42.154 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
17:51:39.542 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 54937 empty documents
17:51:39.621 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [7]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 192509
Number of terms: 188603
Number of postings: 11824971
Number of fields: 0
Number of tokens: 16819835
Field names: []
Positions:   false



In [8]:
display(queries.head())

queries['query_adhoc'] = [el.lower() for el in queries['query_adhoc']]
queries['query_adhoc'] = [remove_punctuation(el) for el in queries['query_adhoc']]
#queries['query_adhoc'] = [remove_stopwords(el) for el in queries['query_adhoc']]
#queries['query_adhoc'] = [porter_stemmer(el) for el in queries['query_adhoc']]

queries['query_desc'] = [el.lower() for el in queries['query_desc']]
queries['query_desc'] = [remove_punctuation(el) for el in queries['query_desc']]
#queries['query_desc'] = [remove_stopwords(el) for el in queries['query_desc']]
#queries['query_desc'] = [porter_stemmer(el) for el in queries['query_desc']]

queries['query_nar'] = [el.lower() for el in queries['query_nar']]
queries['query_nar'] = [remove_punctuation(el) for el in queries['query_nar']]
#queries['query_nar'] = [remove_stopwords(el) for el in queries['query_nar']]
#queries['query_nar'] = [porter_stemmer(el) for el in queries['query_nar']]

queries.head()

,index,qid,query,query_adhoc,query_desc,query_nar
0,0,1,NaN,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,1,2,NaN,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,2,3,NaN,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,3,4,NaN,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,4,5,NaN,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


,index,qid,query,query_adhoc,query_desc,query_nar
0,0,1,NaN,coronavirus origin,what is the origin of covid19,seeking range of information about the sarscov...
1,1,2,NaN,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the sarscov...
2,2,3,NaN,coronavirus immunity,will sarscov2 infected people develop immunity...,seeking studies of immunity developed due to i...
3,3,4,NaN,how do people die from the coronavirus,what causes death from covid19,studies looking at mechanisms of death from co...
4,4,5,NaN,animal models of covid19,what drugs have been active against sarscov or...,papers that describe the results of testing d...


In [9]:
queries_adhoc = queries[['qid', 'query_adhoc']]
queries_adhoc.rename(columns = {"query_adhoc" : "query"}, inplace=True)

queries_desc = queries[['qid', 'query_desc']]
queries_desc.rename(columns = {"query_desc" : "query"}, inplace=True)

queries_narrative = queries[['qid', 'query_nar']]
queries_narrative.rename(columns = {"query_nar" : "query"}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
from pyterrier.measures import *
import onir_pt

#knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='date')
vbert = onir_pt.reranker.from_checkpoint('https://macavaney.us/scibert-medmarco.tar.gz', text_field='title', expected_md5="854966d0b61543ffffa44cea627ab63b")

tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF") % 100
pipeline_tfidf = tfidf >> pt.text.get_text(index, 'title') >> vbert
bm25 = pt.BatchRetrieve(index, wmodel="BM25") % 100
pipeline_bm25 = bm25 >> pt.text.get_text(index, 'title') >> vbert
dir = pt.BatchRetrieve(index, wmodel="DirichletLM") % 100
pipeline_dir = dir >> pt.text.get_text(index, 'title') >> vbert
dph = pt.BatchRetrieve(index, wmodel="DPH") % 100
pipeline_dph = dph >> pt.text.get_text(index, 'title') >> vbert

results = pt.Experiment(
    [tfidf, bm25, dir, dph, pipeline_tfidf, pipeline_bm25, pipeline_dir, pipeline_dph],
    queries_adhoc,
    qrels,
    names = ["tf-idf", "bm25", "DirichletLM", "DPH", "tf-idf + BERT", "bm25 + BERT", "DirichletLM + BERT", "DPH + BERT"],
    eval_metrics=[AP@1000,P@5,P@10, 'map', 'ndcg', R@5, R@10, RR])
display(results)
results.to_csv("first.csv", index = False)

results = pt.Experiment(
    [tfidf, bm25, dir, dph, pipeline_tfidf, pipeline_bm25, pipeline_dir, pipeline_dph],
    queries_desc,
    qrels,
    names = ["tf-idf", "bm25", "DirichletLM", "DPH", "tf-idf + BERT", "bm25 + BERT", "DirichletLM + BERT", "DPH + BERT"],
    eval_metrics=[AP@1000,P@5,P@10, 'map', 'ndcg', R@5, R@10, RR])
display(results)
results.to_csv("second.csv", index = False)

results = pt.Experiment(
    [tfidf, bm25, dir, dph, pipeline_tfidf, pipeline_bm25, pipeline_dir, pipeline_dph],
    queries_narrative,
    qrels,
    names = ["tf-idf", "bm25", "DirichletLM", "DPH", "tf-idf + BERT", "bm25 + BERT", "DirichletLM + BERT", "DPH + BERT"],
    eval_metrics=[AP@1000,P@5,P@10, 'map', 'ndcg', R@5, R@10, RR])
display(results)
results.to_csv("third.csv", index = False)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
config file not found: config


[2022-01-18 17:52:02,112][onir.util.download][DEBUG] downloaded https://macavaney.us/scibert-medmarco.tar.gz [8.31s] [499M] [65.5MB/s] [md5 hash verified]


[2022-01-18 17:52:57,308][onir.util.download][DEBUG] downloaded https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar [51.06s] [411M] [21.9MB/s] [md5 hash verified]


extracting: 411MB [1.04s, 395MB/s]
extracting: 821MB [7.26s, 113MB/s] 


[2022-01-18 17:53:28,812][onir_pt][DEBUG] using GPU (deterministic)
[2022-01-18 17:53:37,976][onir_pt][DEBUG] [starting] batches


...

batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-01-18 17:58:19,986][onir_pt][DEBUG] [finished] batches: [23.96s] [1250it] [52.16it/s]
[2022-01-18 17:58:23,185][onir_pt][DEBUG] using GPU (deterministic)
[2022-01-18 17:58:23,189][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2022-01-18 17:58:46,915][onir_pt][DEBUG] [finished] batches: [23.72s] [1250it] [52.69it/s]


,name,AP@1000,P@5,P@10,map,ndcg,R@5,R@10,RR
0,tf-idf,0.032910,0.440,0.426,0.032910,0.097529,0.004798,0.009124,0.549424
1,bm25,0.033340,0.416,0.420,0.033340,0.098223,0.004634,0.009112,0.565794
2,DirichletLM,0.020440,0.292,0.292,0.020440,0.071928,0.002958,0.005841,0.450707
3,DPH,0.032167,0.436,0.424,0.032167,0.095763,0.004694,0.009409,0.557333
4,tf-idf + BERT,0.043664,0.648,0.590,0.043664,0.111942,0.007411,0.013555,0.756703
5,bm25 + BERT,0.044010,0.644,0.598,0.044010,0.112457,0.007300,0.013630,0.757527
6,DirichletLM + BERT,0.030813,0.608,0.546,0.030813,0.088868,0.006851,0.012115,0.725293
7,DPH + BERT,0.042501,0.680,0.608,0.042501,0.110917,0.007816,0.013772,0.790006
